In [23]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

In [10]:
annotations_df = pd.read_csv("C:/Users/Dell/Downloads/soccer_exercise/soccer_exercise/train/annotations.csv")

In [12]:
output_train_dir = "C:/Users/Dell/Downloads/data/labels/train"
output_val_dir = "C:/Users/Dell/Downloads/data/labels/val"
output_test_dir = "C:/Users/Dell/Downloads/data/labels/test"

In [13]:
#Splitting the training dataset csv file into train and val
train_df, val_df = train_test_split(annotations_df, test_size=0.2, random_state=42)

train_csv_path = os.path.join('C:/Users/Dell/Downloads/data/images','train_annotations.csv')
val_csv_path =os.path.join('C:/Users/Dell/Downloads/data/images','val_annotations.csv')

train_df.to_csv(train_csv_path, index=False)
val_df.to_csv(val_csv_path, index=False)

In [14]:
images_src = 'C:/Users/Dell/Downloads/soccer_exercise/soccer_exercise/train'

train = 'C:/Users/Dell/Downloads/data/images/train'
val = 'C:/Users/Dell/Downloads/data/images/val'



for index, row in train_df.iterrows():
    filename = row['filename']
    image_src = os.path.join(images_src, filename)
    image_dest = os.path.join(train, filename)
    shutil.copy(image_src, image_dest)

for index, row in val_df.iterrows():
    filename = row['filename']
    image_src = os.path.join(images_src, filename)
    image_dest = os.path.join(val, filename)
    shutil.copy(image_src, image_dest)

In [15]:
test_annotations_df=pd.read_csv("C:/Users/Dell/Downloads/data/images/test/annotations.csv")
train_annotations_df=pd.read_csv("C:/Users/Dell/Downloads/data/images/train_annotations.csv")
val_annotations_df=pd.read_csv("C:/Users/Dell/Downloads/data/images/val_annotations.csv")

In [16]:
def convert_to_yolo(image_width, image_height, box):
    x_min, y_min, x_max, y_max = box
    box_width = x_max - x_min
    box_height = y_max - y_min
    x_center = (x_min + x_max) / (2 * image_width)
    y_center = (y_min + y_max) / (2 * image_height)
    yolo_box_width = box_width / image_width
    yolo_box_height = box_height / image_height
    return x_center, y_center, yolo_box_width, yolo_box_height

In [17]:
class_mapping = {
    "player": 0,
    "referee": 1,
    "football": 2
}

In [21]:

for index, row in test_annotations_df.iterrows():
    filename = row['filename']
    width = row['width']
    height = row['height']
    class_label = row['class']
    xmin = row['xmin']
    ymin = row['ymin']
    xmax = row['xmax']
    ymax = row['ymax']
    width = int(width)
    height = int(height)
    bbox = [int(xmin), int(ymin), int(xmax), int(ymax)]

    class_index = class_mapping[class_label]

    yolo_bbox = convert_to_yolo(width, height, bbox)

    yolo_filename = os.path.join(output_test_dir, filename.replace(".jpg", ".txt"))


    with open(yolo_filename, 'a') as yolo_file:
        yolo_line = f'{class_index} ' + ' '.join([str(coord) for coord in yolo_bbox]) + '\n'
        yolo_file.write(yolo_line)



In [4]:
!yolo detect train  model=yolov8s.pt data=C:/Users/Dell/Downloads/data/images/config.yaml epochs=10 imgsz=640 batch= 8 project =C:/Users/Dell/Downloads/training_result name=football

Ultralytics YOLOv8.0.161  Python-3.10.11 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:/Users/Dell/Downloads/data/images/config.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=C:/Users/Dell/Downloads/training_result, name=football, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

In [5]:
!yolo detect predict model=C:/Users/Dell/Downloads/training_result/football3/weights/best.pt conf= 0.55  source=C:/Users/Dell/Downloads/data/images/test save=True classes=2

Ultralytics YOLOv8.0.161  Python-3.10.11 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients

image 1/16 C:\Users\Dell\Downloads\data\images\test\1-fps-2_00001_jpeg_jpg.rf.03aa7dfbdbc3d0a5482ea68a7f9a8a8d.jpg: 384x640 (no detections), 209.0ms
image 2/16 C:\Users\Dell\Downloads\data\images\test\1-fps-2_00014_jpeg_jpg.rf.2d7d526c69632431e3d987a764cbd4e1.jpg: 384x640 (no detections), 197.5ms
image 3/16 C:\Users\Dell\Downloads\data\images\test\1-fps-2_00022_jpeg_jpg.rf.cb129f66c83086ad73336e4c9a33fcc9.jpg: 384x640 1 class_2, 188.1ms
image 4/16 C:\Users\Dell\Downloads\data\images\test\1-fps-2_00025_jpeg_jpg.rf.c9db4792d50f6c60789177a156787289.jpg: 384x640 (no detections), 178.2ms
image 5/16 C:\Users\Dell\Downloads\data\images\test\1-fps-2_00032_jpeg_jpg.rf.c59a43ff2bf0ca9963288bf3bd5f26d2.jpg: 384x640 (no detections), 174.4ms
image 6/16 C:\Users\Dell\Downloads\data\images\test\1-fps-2_00041_jpeg_jpg.rf.258946c9d

In [22]:

src_path = r'runs/detect/predict'
dst_path = r'C:/Users/Dell/Downloads/output'
shutil.copytree(src_path, dst_path)

'C:/Users/Dell/Downloads/output'